# Beispiel: Machine Learning als Sommelier
## Datensatz "Weinqualität"

Wir laden den Datensatz *weinqualitaet.csv* in den Speicher:

In [ ]:
# Datensatz laden
url = "https://raw.githubusercontent.com/troescherw/datasets/master/weinqualitaet.csv"

import pandas as pd
daten = pd.read_csv(url, delimiter=",")
# Löschen von Zeilen mit leeren Einträgen
daten.dropna(inplace=True)

## Beschreibung des Datensatzes

- **type**: white wine or red whine
- **fixed acidity**: most acids involved with wine or fixed or nonvolatile (do not evaporate readily)

- **volatile acidity**: the amount of acetic acid in wine, which at too high of levels can lead to an unpleasant, vinegar taste

- **citric acid**: found in small quantities, citric acid can add 'freshness' and flavor to wines

- **residual sugar**: the amount of sugar remaining after fermentation stops, it's rare to find wines with less than 1 gram/liter and wines with greater than 45 grams/liter are considered sweet

- **chlorides**: the amount of salt in the wine

- **free sulfur dioxide**: the free form of $S0_2$ exists in equilibrium between molecular $S0_2$ (as a dissolved gas) and bisulfite ion; it prevents microbial growth and the oxidation of wine

- **total sulfur**: dioxideamount of free and bound forms of $S0_2$; in low concentrations, $S0_2$ is mostly undetectable in wine, but at free $S0_2$ concentrations over 50 ppm, $S0_2$ becomes evident in the nose and taste of wine

- **density**: the density of water is close to that of water depending on the percent alcohol and sugar content

- **pH**: describes how acidic or basic a wine is on a scale from 0 (very acidic) to 14 (very basic); most wines are between 3-4 on the pH scale

- **sulphates**: a wine additive which can contribute to sulfur dioxide gas ($S0_2$) levels, wich acts as an antimicrobial and antioxidant

- **alcohol**: the percent alcohol content of the wine

- **quality**: output variable (based on sensory data, score between 0 and 10)

In [ ]:
# Datensatz anzeigen
daten

## Einige Statistiken zum Datensatz:

In [ ]:
daten.describe()

Wie viele Weine haben welche Qualitätsstufe erhalten?

In [ ]:
daten["quality"].groupby(daten.quality).count()

Dargestellt als Grafik:

In [ ]:
daten["quality"].groupby(daten.quality).count().plot.bar()

Gibt es Abhängigkeiten zwischen den Features? Wir berechnen die sog. **Korrelationskoeffizienten r**, die immer zwischen den Werten -1 und +1 liegen.

- Nahe -1: Hohe negative Korrelation
- Nahe +1: Hohe positive Korrelation
- Nahe 0: Keine Korrelation


$r = \frac{Cov_{x,y}}{s_x s_y}$


In [ ]:
corrs = daten.corr()

import seaborn as sn
sn.heatmap(corrs)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

plt.scatter(daten.alcohol, daten.density)

Etwas deutlicher zeigt sich der Zusammenhang, wenn wir die Ausreißer entfernen:

In [ ]:
plt.scatter(daten[daten.density<1.01].alcohol/10, daten[daten.density<1.01].density,alpha=.5)
plt.ylim(.98,1.01)
plt.xlabel("Alkoholgehalt")
plt.ylabel("Dichte in g/cm3")
plt.plot()

## Vorhersage-Modell erstellen

Wir wollen anhand der Daten die Weinqualität vorhersagen! Dazu teilen wir den Datensatz in Trainings- und Testdaten auf. Mit 70% der Daten trainieren wir das Modell, mit 30% testen wir das Modell.

In [ ]:
from sklearn.model_selection import train_test_split

X = daten.iloc[:, 0: 12]
X = pd.DataFrame(X, columns = daten.columns[0:12])
X = pd.get_dummies(X)
y = daten.iloc[:, 12]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=0, stratify=y)

Jetzt erstellen wir das Modell mit den Trainings-Daten. Als Modell verwenden wir einen Entscheidungsbaum für Klassifizierung:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=0, criterion="entropy", max_depth=6)
model = model.fit(X_train, y_train)

Wir sagen mit Hilfe des Modells die Weinqualität voraus und verwenden hierfür den Test-Datensatz:

In [ ]:
pred = model.predict(X_test)

Wir vergleichen die vorhergesagte Weinqualität mit der tatsächlichen Weinqualität und stellen dies in einer Tabelle dar:

In [ ]:
pd.crosstab(y_test, pred, rownames=["REFERENZ"], colnames=["VORHERGESAGT"])

Wir setzen die richtig vorhergesagten Werte (die Diagonale in der obigen Tabelle) mit der Gesamtzahl der vorhergesagten Werte ins Verhältnis. Dies gibt die Genauigkeit unseres Modells an, die sog. **Accuracy**:

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred)

Wir erstellen eine Grafik des Baumes:

In [ ]:
%matplotlib inline
from sklearn import tree
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=600)
tree.plot_tree(model, filled=True) 

Welches Feature (=Spalte) hat welche Wichtigkeit auf die Vorhersage?

In [ ]:
pd.Series(model.feature_importances_, index=X.columns).plot.bar()

## Random Forest
Statt nur einen Baum, erstellen wir jetzt gleich eine ganze Menge, genauer gesagt: 500! Für jeden Baum werden zufällig Beobachtungen und Features (Spalten der Tabelle) ausgewählt und jeder Baum für sich trainiert! Somit entscheidet jeder Baum für sich, um welche Weinqualität es sind handelt. Am Ende erfolgt eine "Mehrheitsentscheidung".

Zuerst erstellen wir einen **Random Forest** mit 500 Bäumen.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

model2 = RandomForestClassifier(n_estimators=500)
model2.fit(X_train, y_train)

Jetzt testen wir wieder unser Modell, indem wir eine Vorhersage mit dem Test-Datensatz durchführen:

In [ ]:
pred2 = model2.predict(X_test)

Nun stellen wir wieder die vorhergesagte Weinqualität mit der tatsächlichen in einer Tabelle gegenüber:

In [ ]:
pd.crosstab(y_test, pred2, rownames=["PREDICTED"], colnames=["REFERENCE"])

Wir berechnen wieder die Accuracy:

In [ ]:
accuracy_score(y_test, pred2)

Unser Modell hat nun eine deutlich höhere Accuracy! Ein Random Forest scheint als Modell also besser zu sein als nur ein einzelner Baum!

Wir wollen nun ein anderes Feature vorhersagen: Handelt es sich um einen Rot- oder um einen Weißwein? Unser Datensatz enthält hierfür ein entsprechendes Feature: **type_red** bzw. **type_white**.

In [ ]:
X

Als vorherzusagendes Feature verwenden wir **type_red**: 1 bedeutet Rotwein, 0 bedeutet dann zwangsläufig Weißwein. Wir entfernen daher das Feature **type_white** aus dem Test- und Trainingsdatensatz:

In [ ]:
X = daten.iloc[:, 1: 13]
X = pd.DataFrame(X, columns = daten.columns[1:13])
y = daten.iloc[:, 0]
y = pd.get_dummies(y)["red"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, shuffle=True, random_state=0, stratify=y)

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model3 = DecisionTreeClassifier(random_state=0, criterion="entropy", max_depth=6)
model3 = model.fit(X_train, y_train)

Wir machen wieder eine Vorhersage anhand des Modells:

In [ ]:
pred3 = model3.predict(X_test)

Und geben das Ergebnis im Vergleich zur Realität in einer Tabelle aus:

In [ ]:
pd.crosstab(y_test, pred3, rownames=["PREDICTED"], colnames=["REFERENCE"])

Und berechnen die Accuracy:

In [ ]:
accuracy_score(y_test, pred3)

Welches Feature (=Spalte) hat welche Wichtigkeit auf die Vorhersage?

In [ ]:
pd.Series(model3.feature_importances_, index=X.columns).plot.bar(y)